In [1]:
import face_recognition
import cv2 
import numpy as np
import os
from datetime import datetime 
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import time
from PIL import Image, ImageTk

C:\Users\MI\Desktop\New folder\jupyter_notebook\face_detect\venv\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
# Global variables
known_encodings = []
output_folder = ""
GROUP_IMAGES_FOLDER = "group_image"

def show_live_preview():
    global video_capture, live_preview_running
    if not live_preview_running:
        return

    ret, frame = video_capture.read()
    if not ret:
        return

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(rgb_frame)
    imgtk = ImageTk.PhotoImage(image=img)
 
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    video_label.after(10, show_live_preview)

def live_preview_with_capture():
    def update_frame():
        ret, frame = video_capture.read()
        if not ret:
            log("❌ Failed to capture from webcam.")
            return

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)

        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(rgb_frame, (left, top), (right, bottom), (0, 255, 0), 2)  # Green box

        img = Image.fromarray(rgb_frame)
        imgtk = ImageTk.PhotoImage(image=img)
        video_label.imgtk = imgtk
        video_label.configure(image=imgtk)
        video_label.after(10, update_frame)

    def capture():
        captured_count = 0

        while captured_count < 2:
            ret, frame = video_capture.read()
            if not ret:
                log("❌ Failed to capture image.")
                return

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb_frame)

            if not face_locations:
                log(f"No face detected in capture {captured_count + 1}. Trying again...")
                root.update()
                root.after(1000)  # Wait before trying again
                continue

            encodings = face_recognition.face_encodings(rgb_frame, face_locations)
            if encodings:
                known_encodings.append(encodings[0])
                captured_count += 1
                log(f"✅ Face encoding {captured_count} saved.")
            else:
                log(f"Face detected in capture {captured_count + 1} but encoding failed.")

            root.update()
            root.after(1000)

    
        video_capture.release()
        preview_window.destroy()



   
    preview_window = tk.Toplevel(root)
    preview_window.title("Live Webcam Preview")

    video_label = tk.Label(preview_window)
    video_label.pack()

    capture_button = tk.Button(preview_window, text="📸 Capture", command=capture)
    capture_button.pack(pady=10)

    video_capture = cv2.VideoCapture(0)
    if not video_capture.isOpened():
        messagebox.showerror("Error", "Webcam not accessible.")
        return

    update_frame()

    def on_close():
        video_capture.release()
        preview_window.destroy()

    preview_window.protocol("WM_DELETE_WINDOW", on_close)


def select_output_folder():
    global output_folder
    folder_name = simpledialog.askstring("Folder Name", "Enter a folder name for matched images:")
    if not folder_name:
        return
    output_folder = os.path.join("matched_faces", folder_name)
    os.makedirs(output_folder, exist_ok=True)
    log(f"Output folder set: {output_folder}")

def search_images():
    log("Matching Starts : ")
    root.update_idletasks()
    root.update()
    if not known_encodings:
        log("No known encodings. Please capture face first.")
        root.update_idletasks()
        root.update()
        return

    if not output_folder:
        log("Output folder not set.")
        root.update_idletasks()
        root.update()
        return

    count = 0
    for filename in os.listdir(GROUP_IMAGES_FOLDER):
        if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        image_path = os.path.join(GROUP_IMAGES_FOLDER, filename)
        count += 1
        log(f"[{count}] Processing {filename}...")
        root.update_idletasks()
        root.update()
        group_image = face_recognition.load_image_file(image_path)
        group_locations = face_recognition.face_locations(group_image)
        group_encodings = face_recognition.face_encodings(group_image, group_locations)
        
        

        match_found = False
        for location, encoding in zip(group_locations, group_encodings):
            matches = face_recognition.compare_faces(known_encodings, encoding, tolerance=0.50)
            if any(matches):
                match_found = True
                group_image_bgr = cv2.cvtColor(group_image, cv2.COLOR_RGB2BGR)
                output_path = os.path.join(output_folder, f"{filename}")
                cv2.imwrite(output_path, group_image_bgr)
                log(f"✅ Match found and saved: {output_path}")
                root.update_idletasks()
                root.update()
                break  # Remove this break if you want to save all matches

        if not match_found:
            log(f"❌ No match in: {filename}")
            root.update_idletasks()
            root.update()

    log(f"Matching Ended")
    root.update_idletasks()
    root.update()

        

def log(message):
    log_text.insert(tk.END, f"{datetime.now().strftime('%H:%M:%S')} - {message}\n")
    log_text.see(tk.END)

# Tkinter GUI Setup
root = tk.Tk()
root.title("Face Recognition Tool")

video_label = tk.Label(root)
video_label.pack(pady=10)

tk.Button(root, text="1. Capture Face", width=30, command=live_preview_with_capture).pack(pady=20)
tk.Button(root, text="2. Set Output Folder", width=30, command=select_output_folder).pack(pady=20)
tk.Button(root, text="3. Start Matching", width=30, command=search_images).pack(pady=20)

log_text = tk.Text(root, height=20, width=60)
log_text.pack(pady=10)

root.mainloop()
